## Примеры SQL запросов

Составим таблицу из полей:
    name_of_fund — название фонда;
    name_of_company — название компании;
    amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
WITH
cm AS (SELECT id,
              name AS name_of_company
       FROM company
       WHERE milestones > 6),
       
fr AS (SELECT company_id,
              id,
              raised_amount
       FROM funding_round
       WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN '2012' AND '2013'),
       
inv AS (SELECT funding_round_id,
               fund_id
       FROM investment),
       
fun AS (SELECT id,
               name
        FROM fund)
        
SELECT fun.name AS name_of_fund,
       cm.name_of_company AS name_of_company,
       fr.raised_amount AS amount
FROM cm
JOIN fr ON cm.id=fr.company_id
JOIN inv ON fr.id=inv.funding_round_id
JOIN fun ON inv.fund_id= fun.id


Отберем данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируем данные по номеру месяца и получим таблицу, в которой будут поля: 
    номер месяца, в котором проходили раунды; 
    количество уникальных названий фондов из США, которые инвестировали в этом месяце; 
    количество компаний, купленных за этот месяц; 
    общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
i AS (
    SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month,
           COUNT(DISTINCT (i.fund_id)) AS fund_number
     FROM funding_round AS fr
     LEFT JOIN investment AS i ON fr.id = i.funding_round_id
     LEFT JOIN fund AS f ON i.fund_id = f.id
     WHERE f.country_code = 'USA'
     AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
     GROUP BY EXTRACT(MONTH FROM CAST(fr.funded_at AS date))
),
ta AS (
       SELECT EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS month,
              COUNT(a.acquired_company_id) AS bought_cm,
              SUM(a.price_amount) AS amount
      FROM  acquisition AS a
      WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS date)) BETWEEN 2010 AND 2013
      GROUP BY MONTH)
SELECT i.month,
       i.fund_number,
       ta.bought_cm,
       ta.amount
FROM i inner join ta ON i.month=ta.month

Составим сводную таблицу и выведим среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год будут в отдельном поле. Отсортируем таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH

i AS (SELECT country_code AS country,
             AVG(funding_total) AS funding_total_2011
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2011'
      GROUP BY country_code),
      
j AS (SELECT country_code AS country,
             AVG(funding_total) AS funding_total_2012
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2012'
      GROUP BY country_code),
      
l AS (SELECT country_code AS country,
             AVG(funding_total) AS funding_total_2013
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2013'
      GROUP BY country_code)
      
SELECT i.country,
       i.funding_total_2011,
       j.funding_total_2012,
       l.funding_total_2013
FROM i
JOIN j ON i.country=j.country
JOIN l ON j.country=l.country
ORDER BY i.funding_total_2011 DESC

Сформируем статистику по категориям фильмов. Отобразим в итоговой таблице два поля:
    название категории;
    число фильмов из этой категории.
Фильмы для второго поля отберем по условию. Посчитаем фильмы только с теми актёрами и актрисами, которые больше семи раз снимались в фильмах, вышедших после 2013 года. 
Назовем поля name_category и total_films соответственно. Отсортируем таблицу по количеству фильмов от большего к меньшему, а затем по полю с названием категории в лексикографическом порядке.

In [ ]:
SELECT c.name as name_category,
       count(DISTINCT movies.film_id) as total_films
FROM           
     (SELECT movie.film_id 
              FROM movie inner join film_category fc on movie.film_id=fc.film_id
              INNER JOIN film_actor fa on fc.film_id=fa.film_id
      WHERE fa.actor_id in  (SELECT fa.actor_id
                                FROM movie INNER JOIN film_category fc on movie.film_id=fc.film_id
                                           INNER JOIN film_actor fa on fc.film_id=fa.film_id
                                WHERE release_year>2013
                                GROUP BY fa.actor_id 
                                HAVING count(movie.film_id)>7)
     ) movies     
        INNER JOIN film_category fc on movies.film_id=fc.film_id
        INNER JOIN category c on fc.category_id=c.category_id
GROUP BY name_category 
ORDER BY total_films desc, name_category 

Определим, летом какого года общая выручка в магазине была максимальной. Затем проанализируем данные за этот год по странам. Выгрузим таблицу с полями:
    country — название страны;
    total_invoice — число заказов, оформленных в этой стране в тот год, когда общая выручка за лето была максимальной;
    total_customer — число клиентов, зарегистрированных в этой стране.
Отсортируем таблицу по убыванию значений в поле total_invoice, а затем добавим сортировку по названию страны в лексикографическом порядке.

In [ ]:
SELECT l.country,
       COUNT(j.customer_id) AS total_invoice,
       l.total_customer
FROM  
   (SELECT customer_id
   FROM invoice
   WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) IN (SELECT EXTRACT(YEAR FROM CAST(invoice_date AS date)) AS date_all
                                                              FROM invoice 
                                                              WHERE EXTRACT(MONTH FROM CAST(invoice_date AS date)) BETWEEN '6' AND '8'
                                                              GROUP BY date_all
                                                              ORDER BY SUM(total) DESC
                                                              LIMIT 1)) AS i
JOIN
   (SELECT customer_id,
           billing_country
    FROM invoice
    GROUP by customer_id, billing_country) AS j ON i.customer_id=j.customer_id
JOIN
   (SELECT country,
           COUNT(customer_id) AS total_customer
    FROM client as cl
    GROUP BY country) AS l ON j.billing_country=l.country
GROUP BY l.country, l.total_customer
ORDER BY total_invoice DESC, country